<a href="https://colab.research.google.com/github/rieko0903/NaturalLanguageProcessing01/blob/test02/natural_language_processing01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [ ]:
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.2 MB/s 


# import

In [42]:
import tensorflow as tf
import glob
import logging
import os
import re
import urllib
import warnings
import zipfile

import gensim
import numpy as np
import tensorflow as tf

from gensim.models.word2vec import Word2Vec
from janome.tokenizer import Tokenizer

In [43]:
URL = 'https://www.aozora.gr.jp/cards/000879/files/4872_ruby_20864.zip'

# 文章の準備

## 文章のダウンロード
*   os内にフォルダを作成、zipファイルを展開
*   中身のテキストを抽出
*   データファイルをダウンロード

In [44]:
def download(url):
    # データファイルをダウンロードする
    zip_file = re.split(r'/', url)[-1]

    if not os.path.exists(zip_file):
        print('Download URL')
        print('URL:',url)
        urllib.request.urlretrieve(url, zip_file)
    else:
        print('Download File exists')

    # フォルダの生成
    dir, ext = os.path.splitext(zip_file)
    if not os.path.exists(dir):
        os.makedirs(dir)

    # zipファイルの展開
    zip_obj = zipfile.ZipFile(zip_file, 'r')
    zip_obj.extractall(dir)
    zip_obj.close()

    # zipファイルの削除
    os.remove(zip_file)

    # テキストファイルの抽出
    path = os.path.join(dir,'*.txt')
    list = glob.glob(path)
    return list[0]

# 文書の整形
* ルビやいらない文字（本文以外）の削除

In [45]:
def convert(download_text):
    binarydata = open(download_text, 'rb').read()
    text = binarydata.decode('shift_jis')
 
    # ルビ、注釈などの除去
    text = re.split(r'\-{5,}', text)[2]
    text = re.split(r'底本：', text)[0]
    text = re.sub(r'《.+?》', '', text)
    text = re.sub(r'［＃.+?］', '', text)
    text = text.strip()
    return text

In [66]:
t = Tokenizer(wakati=True)
def MakeSet():
    download_text = download(URL)
    text = convert(download_text)
    print(text)
    print("----------------")
    wakati = t.tokenize(text)
    for i in wakati:
        print(i, end=" ")
        if i =="。":
            print("")
    vocab = sorted(set(text))


In [ ]:
class AozoraText:
  def __init__(self, url):
    download_text = download(URL)
    self.text = convert(download_text)
    self.wakati, self.word2idx, self.idx2word, self.corpus = preprocess(self.text)

  def make_writer_dataset(self, batch_size, seq_length):
    data = tf.data.Dataset.from_tensor_slices(self.corpus)
    sequences = data.batch(seq_length+1, drop_remainder=True)
    dataset = sequences.map(split_input_target)
    return dataset.shuffle(buffer_size=10000).batch(batch_size=batch_size, drop_remainder=True)

In [67]:
if __name__ == "__main__":
    MakeSet()

Download URL
URL: https://www.aozora.gr.jp/cards/000879/files/4872_ruby_20864.zip
子供の時の愛読書は「西遊記」が第一である。これ等は今日でも僕の愛読書である。比喩談としてこれほどの傑作は、西洋には一つもないであらうと思ふ。名高いバンヤンの「天路歴程」なども到底この「西遊記」の敵ではない。それから「水滸伝」も愛読書の一つである。これも今以て愛読してゐる。一時は「水滸伝」の中の一百八人の豪傑の名前を悉く諳記してゐたことがある。その時分でも押川春浪氏の冒険小説や何かよりもこの「水滸伝」だの「西遊記」だのといふ方が遥かに僕に面白かつた。
　中学へ入学前から徳富蘆花氏の「自然と人生」や樗牛の「平家雑感」や小島烏水氏の「日本山水論」を愛読した。同時に、夏目さんの「猫」や鏡花氏の「風流線」や緑雨の「あられ酒」を愛読した。だから人の事は笑へない。僕にも「文章倶楽部」の「青年文士録」の中にあるやうな「トルストイ、坪内士行、大町桂月」時代があつた。
　中学を卒業してから色んな本を読んだけれども、特に愛読した本といふものはないが、概して云ふと、ワイルドとかゴーチエとかいふやうな絢爛とした小説が好きであつた。それは僕の気質からも来てゐるであらうけれども、一つは慥かに日本の自然主義的な小説に厭きた反動であらうと思ふ。ところが、高等学校を卒業する前後から、どういふものか趣味や物の見方に大きな曲折が起つて、前に言つたワイルドとかゴーチエとかといふ作家のものがひどくいやになつた。ストリンドベルクなどに傾倒したのはこの頃である。その時分の僕の心持からいふと、ミケエロ・アンヂエロ風な力を持つてゐない芸術はすべて瓦礫のやうに感じられた。これは当時読んだ「ジヤンクリストフ」などの影響であつたらうと思ふ。
　さういふ心持が大学を卒業する後までも続いたが、段々燃えるやうな力の崇拝もうすらいで、一年前から静かな力のある書物に最も心を惹かれるやうになつてゐる。但、静かなと言つてもたゞ静かだけでも力のないものには余り興味がない。スタンダールやメリメエや日本物で西鶴などの小説はこの点で今の僕には面白くもあり、又ためにもなる本である。
　序ながら附け加へておくが、此間「ジヤンクリストフ」を出して読んで見たが、昔ほど感興が乗らなかつた。あの